In [64]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaIoBaseDownload
import io
import csv

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive']

def main():
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=50001)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)
    sheets = build('sheets', 'v4', credentials=creds)

    # Call the Drive v3 API
    results = service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])
    
    if not items:
        print('No files found.')
    else:
        print('Files:')
        for item in items:
            print(u'{0} ({1})'.format(item['name'], item['id']))
    
    file_id = '1RZV-dMW4RRIE8HjvzAHgtK2kmr7wCoYAQMLroHxvjKc'
    #request = service.files().export_media(fileId=file_id,
    #                                         mimeType='application/x-vnd.oasis.opendocument.spreadsheet')
    #fh = io.FileIO('salida.csv', 'wb')
    #downloader = MediaIoBaseDownload(fh, request)
    #done = False
    #while done is False:
    #    status, done = downloader.next_chunk()
    #    print(int(status.progress() * 100))
    
    download_sheet_to_csv(sheets, file_id, 'Respuestas de formulario 1')

if __name__ == '__main__':
    main()

Files:
Noticias - Vigilancia Tecnológica - Servicio de Internacionalización e Innovación (respuestas) (1RZV-dMW4RRIE8HjvzAHgtK2kmr7wCoYAQMLroHxvjKc)
Noticias - Vigilancia Tecnológica - Servicio de Internacionalización e Innovación (1H1g_4CnKnW5X_UKtBDRzXo9C2igpjLmNz3Hg13VrfRo)
20200530_Proyecto_AnalisisRRSS.pdf (13NrWIJzjkQGM0EiT_ncmYrQhl_-_y5hm)
Getting started (0B8fzYFAsjJq0c3RhcnRlcl9maWxl)
Successfully downloaded Respuestas de formulario 1.csv


In [59]:
def download_sheet_to_csv(sheets_instance, spreadsheet_id, sheet_name):
    result = sheets_instance.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=sheet_name).execute()
    output_file = f'{sheet_name}.csv'

    with open(output_file, 'w') as f:
        writer = csv.writer(f)
        writer.writerows(result.get('values'))

    f.close()

    print(f'Successfully downloaded {sheet_name}.csv')